In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

데이터 출처 : https://www.kaggle.com/datasets/mandysia/obesity-dataset-cleaned-and-data-sinthetic 후처리

데이터 링크 : https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/31/adp_31_1_obesity_v2.csv

데이터 설명 : 각 환자의 의료정보이다. NObeyesdad를 종속변수로 하는 분류모델을 만드려고 한다.

- id: unique id for each row
- Gender: sex - male or female
- Age: age
- Height: height
- Weight: weight
- family_history_with_overweight: Has a family member suffered or suffers f from overweight? - yes or no
- FAVC: Frequent consumption of high caloric food - yes or no
- FCVC: Frequency of consumption of vegetables - Never, Sometimes, Always
- NCP: Number of main meals - 1, 2, 3, 4
- CAEC: Consumption of food between meals - No, Sometimes, Frequently, Always
- SMOKE: Do you smoke - yes o no
- CH2O: Consumption of water daily - Less than a litter, between 1 and 2 l, more than 2 l
- SCC: Calories consumption monitoring - yes or no
- FAF: Physical activity frequency - 0, 1 to 2, 2 to 4, 4 to 5
- TUE: Time using technology devices - 0 to 2, 3 to 5, >5
- CALC: Consumption of alcohol - no, sometimes, frequently, always
- MTRANS: Transportation used - automobile, motorbike, bike, public_transportation, walking
- BMI: Body mass index

(종속변수)
NObeyesdad: Type of obesity - overweight-level_i, obesity_type_i, obesity_type_ii, obesity_type_iii

In [ ]:
import pandas as pd 
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/31/adp_31_1_obesity_v2.csv",index_col=0)
df.head()

1-1 EDA & 결측치 및 이상치를 판단하고 처리하라


1-2 데이터 전처리 기법 2가지를 설명하고 주어진 데이터에 적용시 어떤 효과가 있는지 설명하라


1-3 피쳐 엔지니어링을 통해 파생변수 1개를 생성하고 그 이유를 말하라



2-1 앙상블을 제외한 분류 모델 3가지 구축 및 결과 비교 및 설명하라

2-2 2-1에서 사용한 모델 중 하나를 골라 그리드 서치를 통해서 파라미터 튜닝 및 분류 모델 성능 평가 (precision ,recall)

2-3 2-1의 3가지 모델을 soft voting을 이용하여 모델링 한 결과와 2-2과 비교하라



3-1 하나의 모델을 선정하여 Drop Column Importance 방식으로 각 변수 별로 전체 컬럼을 포함한 모델과의 recall 값의 차이를 계산하고 시각화 하라


데이터 출처 : 자체제작

데이터 링크 : https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/31/adp_31_2_v2.csv

데이터 설명 : 중고등학생 건강검진 데이터

In [3]:
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/31/adp_31_2_v2.csv')
df.head()


,ID,키,weight,생년월일,건강검진일,공학여부,채소섭취정도,아침식사여부,일주일운동시간,수면시간,성별
0,ID_4135,169.01,65.47,20041003,2020_11_15,1,2,1,4.4,8.3,남성
1,ID_3289,181.62,69.36,19970725,2014_11_20,0,3,0,4.4,6.9,남성
2,ID_1847,160.89,65.12,20020921,2020_01_28,1,1,1,1.7,9.6,여성
3,ID_4785,162.21,62.28,20020106,2018_09_27,1,4,0,5.1,6.8,남성
4,ID_5693,159.13,54.11,19980708,2015_03_03,0,4,1,0.3,8.5,여성


아래의 기준으로 전처리를 하여 적정 체중 여부 컬럼을 생성하고 BMI를 5단위로 구분하여 와 적정 체중여부에 대한 빈도 표를 만들어라

bmi 계산 - Bmi = 몸무게(kg)/(키(m)**2)


만나이 구하기 - 건강검진을 받았던 날을 기준으로 생년월일과 일수 차이가 16년 364일 이하인 경우 만 16세 그 이상의 경우 만 17로 분류하라 - 윤년 등은 고려하지 않는다. 햇수로 16년 + 일수로 364일이 기준이다

적정 체중 여부 (BMI가 아래 구간에 들어올 경우)
17세 남자 : 21.03이상 23.21 미만
17세 여자 : 20.03이상 22.39 미만
16세 남자 : 21.18이상 23.45 미만
16세 여자 : 19.61이상 21.74 미만


4-2 4-1에서 구한 적정 체중 여부와 나머지 컬럼(공학여부, 아침식사여부, 일주일운동시간, 채소섭취정도, 수면시간, 성별) 이 독립적인지 통계적으로 확인하라

4-3 4-2에서 유의한 변수들만 가지고 적정 체중 여부를 예측하는 모델을 구현하고 성능 평가 및 해석을 하라 (로지스틱회귀 ,xgb)

로지스틱회귀 모델은 오즈비를 구하라 xgb의 경우 각 피쳐중요도를 확인하고 예측에 영향을 가장 미치는 변수를 확인하라

4-4 4-3 두 모델의 roc-auc 그래프를 하나의 그래프에 겹쳐 그려라

5 평균이 𝜃이고 분산이 100인 정규분포의 사전분포가 100, 256일때 120의 값을 가지는 데이터가 있을 경우 사후평균은?

데이터 설명

데이터 출처 : https://www.kaggle.com/datasets/yasserh/advertising-sales-dataset 후처리

데이터 링크 : https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/31/adp_31_5_advertising.csv

데이터 설명 : TV, Radio, Newspaper에 각각 광고비(달러)를 다르게 했을때 매출액 (Sales , 밀리언달러)를 나타내는 데이터

종속변수 : Sales

6-1 회귀 모델링 후 유의하지 않는변수 파악

6-2 변수 선택시 먼저 제거 될 변수 및 근거 제시

6-3 VIF를 통한 다중공선성 진단

데이터 링크 : https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/31/adp_31_7.csv

데이터 설명 : A,B,C,D,E 영업사원의 각 계약 성사 유무 (1:계약 , 0:미계약) 를 나타낸 데이터이다. 영업사원의 평균 계약 성사 건수는 같은지 통계 검정하라


In [6]:
import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/31/adp_31_7.csv',index_col = 0)
df.head()


,A,B,C,D,E
contract_1,1,0,1,1,1
contract_2,0,1,0,1,0
contract_3,1,0,0,0,0
contract_4,0,1,1,1,1
contract_5,0,1,0,1,0


8 유기견이 하루에 2.2마리 발생한다.

8-1 한마리도 한 버려질 확률

8-2 적어도 2마리가 버려질 확률

